### IMPORTANT: Please make sure that your are using the bash kernel to run this notebook.
#### (Do this at the beginning of every session) ###

In [ ]:
### Set up variables storing the location of our data
### The proper way to load your variables is with the ~/.bashrc command, but this is very slow in iPython 
export SUNETID="$(whoami)"
export WORK_DIR="/scratch/${SUNETID}"
export DATA_DIR="${WORK_DIR}/data"
[[ ! -d ${WORK_DIR}/data ]] && mkdir "${WORK_DIR}/data"
export SRC_DIR="${WORK_DIR}/src"
[[ ! -d ${WORK_DIR}/src ]] && mkdir -p "${WORK_DIR}/src"
export METADATA_DIR="/metadata"
export AGGREGATE_DATA_DIR="/data"
export AGGREGATE_ANALYSIS_DIR="/outputs"
export YEAST_DIR="/saccer3"
export TMP="${WORK_DIR}/tmp"
export TEMP=$TMP
export TMPDIR=$TMP
[[ ! -d ${TMP} ]] && mkdir -p "${TMP}"

source /etc/profile.d/modules.sh
module load ucsc_tools

In [ ]:
cd $WORK_DIR

In [ ]:
export experiment="0min_HOG1" #select your experiment 


In [ ]:
#We will begin by converting the TN5-shifted tagAlign file of reads to a bigwig file for use with the deeptools package
export tagAlign=/outputs/croo/$experiment/align/pooled-rep/$experiment\_1_R1.merged.nodup.tn5.pooled.tagAlign.gz

In [ ]:
echo $tagAlign

In [ ]:
#sort the tagAlign
bedtools sort -i $tagAlign > $experiment.tagAlign.sorted.bed
#compute genome coverage
bedtools genomecov -i $experiment.tagAlign.sorted.bed -bg -g $YEAST_DIR/sacCer3.chrom.sizes > $experiment.bedGraph
#convert to bigwig 
bedGraphToBigWig $experiment.bedGraph  $YEAST_DIR/sacCer3.chrom.sizes $experiment.bw

We now use the [deeptools computeMatrix command](https://deeptools.readthedocs.io/en/develop/content/tools/computeMatrix.html) to compute TSS enrichment

In [ ]:
computeMatrix --help

In [ ]:
computeMatrix reference-point \
              -R $YEAST_DIR/yeast_tss_coords.bed \
              -S $experiment.bw \
              --referencePoint center \
              -o $experiment.TSS.mat \
              -b 1000 \
              -a 1000 \
              -bs 10


In [ ]:
#plot TSS enrichment heatmap 
plotHeatmap --matrixFile $experiment.TSS.mat -o $experiment.TSS.enrich.png

The resulting plot can be visualized in teh browser, in your workdir: http://1.gentc.net/outputs/[yourname]/[experiment].TSS.enrich.png, and should look something like this: 

![yeast_tss](images/0min_HOG1.TSS.enrich.png)